In [42]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [43]:
!docker build --no-cache -t huggingface-sagemaker-pytorch-training-detectron2 .

Sending build context to Docker daemon  219.6kB
Step 1/6 : FROM 636218042492.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-training-detectron2:latest
 ---> db117cd6691d
Step 2/6 : RUN pip3 install --upgrade datasets==2.2.2 pathos seqeval
 ---> Running in eb1e5fe9103a
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.8/346.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.7/81.7 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 28.0 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=953fafad676538899834959a2bff98635b2b5bb5d958481a43f1

In [44]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
# role = "arn:aws:iam::636218042492:role/service-role/AmazonSageMaker-ExecutionRole-20220829T114489"
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::636218042492:role/service-role/AmazonSageMaker-ExecutionRole-20220829T114489
sagemaker bucket: sagemaker-us-east-1-636218042492
sagemaker session region: us-east-1


In [45]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
print(torchvision.__version__)

1.10.0 True
0.11.1


In [46]:
from sagemaker.huggingface import HuggingFace
import time

print(sagemaker.__version__)

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 1,                          # number of training epochs
                 'train_batch_size': 4,               # batch size for training
                 'eval_batch_size': 2,                # batch size for evaluation
#                  'learning_rate': 5e-5,                # learning rate used during training
                 'model_id':'microsoft/layoutlmv2-base-uncased', # pre-trained model
#                  'fp16': True,                         # Whether to use 16-bit (mixed) precision training
                }

2.108.0


In [58]:
# define Training Job Name 
job_name = f'huggingface-workshop-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'train.py',        # fine-tuning script used in training jon
#     source_dir           = './',       # directory where fine-tuning script is stored
    # instance_type        = 'ml.p3.2xlarge',   # instances type used for the training job
    instance_type        = 'local_gpu',
    instance_count       = 1,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    # transformers_version = '4.17.0',           # the transformers version used in the training job
    # pytorch_version      = '1.10.2',           # the pytorch_version version used in the training job
    py_version           = 'py38',            # the python version used in the training job
    image_uri            = 'huggingface-sagemaker-pytorch-training-detectron2',
    hyperparameters      = hyperparameters,   # the hyperparameter used for running the training job
)

In [59]:
huggingface_estimator.training_image_uri()
huggingface_estimator.instance_type
job_name

'huggingface-workshop-2022-09-06-05-22-38'

In [60]:
training_input_path = f's3://{sess.default_bucket()}/dataset-pp/train'
test_input_path = f's3://{sess.default_bucket()}/dataset-pp/test'

# define a data input dictionary with our uploaded s3 uris
data = {
    'train': training_input_path,
    'test': test_input_path
}

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=True)

Creating kvbldx1s3x-algo-1-pr0d0 ... 
Creating kvbldx1s3x-algo-1-pr0d0 ... done
Attaching to kvbldx1s3x-algo-1-pr0d0
kvbldx1s3x-algo-1-pr0d0 | 2022-09-06 05:22:49,737 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
kvbldx1s3x-algo-1-pr0d0 | 2022-09-06 05:22:49,766 sagemaker-training-toolkit INFO     instance_groups entry not present in resource_config
kvbldx1s3x-algo-1-pr0d0 | 2022-09-06 05:22:49,767 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
kvbldx1s3x-algo-1-pr0d0 | 2022-09-06 05:22:49,769 sagemaker_pytorch_container.training INFO     Invoking user training script.
kvbldx1s3x-algo-1-pr0d0 | 2022-09-06 05:22:49,800 sagemaker-training-toolkit INFO     instance_groups entry not present in resource_config
kvbldx1s3x-algo-1-pr0d0 | 2022-09-06 05:22:49,829 sagemaker-training-toolkit INFO     instance_groups entry not present in resource_config
kvbldx1s3x-algo-1-pr0d0 | 2022-09-06 05:22:49,859 sagemaker

OSError: [Errno 28] No space left on device

# Deploy to Inference

In [62]:
predictor = huggingface_estimator.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")

ValueError: Estimator is not associated with a training job

# Upload image to AWS

In [63]:
%%sh

# Specify an algorithm name
algorithm_name=huggingface-sagemaker-pytorch-training-detectron2

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Log into Docker
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  250.4kB
Step 1/6 : FROM 636218042492.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-training-detectron2:latest
 ---> db117cd6691d
Step 2/6 : RUN pip3 install --upgrade datasets==2.2.2 pathos seqeval
 ---> Using cache
 ---> 38ab1a72e4ef
Step 3/6 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 699e24bae726
Step 4/6 : ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code
 ---> Using cache
 ---> a789df7937ea
Step 5/6 : COPY train.py /opt/ml/code/train.py
 ---> Using cache
 ---> 938f3a00f931
Step 6/6 : ENV SAGEMAKER_PROGRAM train.py
 ---> Using cache
 ---> b81acacdb886
Successfully built b81acacdb886
Successfully tagged huggingface-sagemaker-pytorch-training-detectron2:latest
The push refers to repository [636218042492.dkr.ecr.us-east-1.amazonaws.com/huggingface-sagemaker-pytorch-training-detectron2]
3085eb796ab5: Preparing
64efc7958186: Preparing
80706fa6daba: Preparing
7c0fd75aa551: Preparing
8df935dd71df: Preparing
32b987de1bba

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

